# Impact Analyzer

In [ ]:
import polars as pl
import pathlib
import pdstools
import polars.selectors as cs

# Data

Export the VBD Actuals dataset (in production) or VBD Scenario Planner Actuals (in BOE) from Pega Dev or App Studio.

In [ ]:
# vbd_export_path = pathlib.Path(
#     "~/Downloads/Data-pxStrategyResult_ActualsExport_20240221T204009_GMT.zip"
# ).expanduser()

# vbd_export_path = pathlib.Path(
#     "~/Library/CloudStorage/OneDrive-PegasystemsInc/AI Chapter/projects/Impact Analyzer/Lmms-22-02-2023/Data-pxStrategyResult_Actuals_20230222T154840_GMT.zip"
# ).expanduser()

vbd_export_path = pathlib.Path(
    "~/Downloads/Data-pxStrategyResult_ScenarioPlannerActuals_20240202T144246_GMT.zip"
).expanduser()

vbd_export = pdstools.readDSExport(vbd_export_path).with_columns(
    pdstools.cdh_utils.parsePegaDateTimeFormats("pxOutcomeTime").dt.date(),
)

In [ ]:
cols = vbd_export.columns
cols.sort()
cols

Fix up some data. **FinalPropensity** is not always in IH, only by customization or starting from v xxx onwards.

In [ ]:
if "FinalPropensity" not in vbd_export.columns:
    vbd_export = vbd_export.with_columns(pl.lit(None).alias("FinalPropensity"))

# Control Groups in Impact Analyzer

IA uses **pyReason**, **MktType**, **MktValue** and **ModelControlGroup** to define the various experiments. For the standard NBA decisions (no experiment), values are left empty (null). 

Prior to Impact Analyzer, or when turned off, Predictions from Prediction Studio manage two groups through the **ModelControlGroup** property. A value of **Test** is used for model driven arbitration, **Control** for the random control group (defaults to 2%).

When IA is on, the distinct values from just **MktValue** are sufficient to identify the different experiments.


In [ ]:
default_ia_experiments = pl.DataFrame(
    {
        "Experiment": [
            "NBA vs Random",
            "NBA vs Propensity Only",
            "NBA vs No Levers",
            "NBA vs Only Eligibility Rules",
            "Prioritization w Random p",
            "Prioritization w ADM only",
            "Full NBA",
        ],
        "MktType": [
            "NBAPrioritization",
            "PropensityPriority",
            "LeverPriority",
            "EngagementPolicy",
            "ModelControl",
            "ModelControl",
            None,
        ],
        "MktValue": [
            "NBAHealth_NBAPrioritization",
            "NBAHealth_PropensityPriority",
            "NBAHealth_LeverPriority",
            "NBAHealth_EngagementPolicy",
            "NBAHealth_ModelControl_1",
            "NBAHealth_ModelControl_2",
            None,
        ],
        "pyReason": [
            "Control",
            "Control",
            "Control",
            "Control",
            "Control",
            "Control",
            None,
        ],
        "Description": [
            "Random eligible action (all engagement policies but randomly prioritized)",
            "Prioritized with propensity only (no V, C or L)",
            "Prioritized with no levers (only p, V and C)",
            "Only Eligibility policies applied (no Applicability or Suitability, and prioritized with pVCL)",
            "Prioritized with Random (p) only",
            "Prioritized with Adaptive model (p) only",
            "Arbitrated with your full NBA as configured",
        ],
    }
)
default_ia_experiments.to_pandas().style.hide(axis="index")

# No-Action 

The usage of "Default" issues and groups indicates that there is no action. These need to be filtered out for proper reporting.

TODO: exclude from analysis

In [ ]:
vbd_export.group_by(["pyChannel", "pyDirection", "pyIssue", "pyGroup"]).agg(
    pl.count().alias("VBD Records"),
    pl.col("AggregateCount").cast(pl.Int64).sum().alias("Actions"),
).with_columns(
    (pl.col("VBD Records") / pl.sum("VBD Records"))
    .over(["pyChannel", "pyDirection"])  # Percentages relative to channel
    .alias("VBD Records Percentage (per channel)"),
    (pl.col("Actions") / pl.sum("Actions"))
    .over(["pyChannel", "pyDirection"])  # Percentages relative to channel
    .alias("Actions Percentage (per channel)"),
).filter((pl.col("pyIssue") == "Default") | (pl.col("pyGroup") == "Default")).collect()

# Lookback period

In [ ]:
lookback = "-90d"  # "-1mo", "-2y", "-2w" etc https://docs.pola.rs/py-polars/html/reference/expressions/api/polars.Expr.dt.offset_by.html

lookback_time = (
    vbd_export.select(pl.col("pxOutcomeTime").max().dt.offset_by(lookback))
    .collect()
    .item()
)

lookback_time

# Impact Analyzer counts by Channel

In [ ]:
optional_mcg = (
    ["ModelControlGroup"] if "ModelControlGroup" in vbd_export.columns else []
)

ia_summary_by_channel = (
    vbd_export.filter(pl.col("pxOutcomeTime") >= lookback_time)
    .group_by(
        [
            "pyChannel",
            "pyDirection",
            "MktType",
            "MktValue",
            "pyReason",
            "pyOutcome",
        ]
        + optional_mcg
    )
    .agg(
        pl.col("pxOutcomeTime").max().alias("Most Recent Update"),
        pl.count().alias("VBD Records"),
        pl.col("AggregateCount").cast(pl.Int64).sum().alias("Actions"),
        pl.sum("FinalPropensity"),
        pl.sum("pyPropensity"),
        pl.sum("pxPriority"),
    )
    .with_columns(
        (pl.col("VBD Records") / pl.sum("VBD Records"))
        .over(["pyChannel", "pyDirection"])  # Percentages relative to channel
        .alias("VBD Records Percentage (per channel)"),
        (pl.col("Actions") / pl.sum("Actions"))
        .over(["pyChannel", "pyDirection"])  # Percentages relative to channel
        .alias("Actions Percentage (per channel)"),
        (pl.col("FinalPropensity") / pl.col("Actions")).alias("Avg FinalPropensity"),
        (pl.col("pyPropensity") / pl.col("Actions")).alias("Avg pyPropensity"),
        (pl.col("pxPriority") / pl.col("Actions")).alias("Avg pxPriority"),
    )
    .drop(["FinalPropensity", "pyPropensity", "pxPriority"])
    .join(
        default_ia_experiments.lazy(),
        how="left",
        on=["MktValue", "MktType", "pyReason"],
        join_nulls=True,
    )
    .sort(
        [
            "pyChannel",
            "pyDirection",
            "Experiment",
        ]
        + optional_mcg,
        nulls_last=True,
    )
)


def highlight(s):
    if s.Experiment is None:
        return ["background-color: orange"] * len(s)
    else:
        return ["background-color: white"] * len(s)


ia_summary_by_channel_formatted = (
    ia_summary_by_channel.filter(pl.col("pyChannel") == "Web")
    .collect()
    .to_pandas()
    .style.format(
        {
            "Avg FinalPropensity": "{:.2%}",
            "Avg pyPropensity": "{:.2%}",
            "Avg pxPriority": "{:.3f}",
            "VBD Records Percentage (per channel)": "{:.2%}",
            "Actions Percentage (per channel)": "{:.2%}",
            "Most Recent Update": "{:%d %b '%y}",
        }
    )
    .hide(axis="index")
    .hide(["Description", "VBD Records Percentage (per channel)"], axis="columns")
    .set_caption("Experiment Summary:")
    .apply(highlight, axis=1)
)

ia_summary_by_channel_formatted

# KPIs per Channel

In [ ]:
# ia_summary_by_channel.collect().pivot("pyOutcome")
# xxx = set(ia_summary_by_channel.columns)
# xxx.remove('pyOutcome')
import pandas as pd

group_by_cols = [
    "pyChannel",
    "pyDirection",
    # "MktType",
    "Experiment",
    "MktValue",
    # "pyReason",
] + optional_mcg

engagement_overview = (
    ia_summary_by_channel.collect()
    .pivot(
        index=group_by_cols,
        columns="pyOutcome",
        values="Actions",
        aggregate_function="sum",
        sort_columns=True,
    )
    .with_columns(cs.numeric().fill_null(0))
)

positive_labels = [
    label
    for label in ["Clicked", "Accept", "Accepted"]
    if label in engagement_overview.columns
]
negative_labels = [
    label for label in ["Impression", "Viewed"] if label in engagement_overview.columns
]

pos_expr = pl.lit(0.0)
for label in positive_labels:
    pos_expr = pos_expr + pl.col(label)
neg_expr = pl.lit(0.0)
for label in negative_labels:
    neg_expr = neg_expr + pl.col(label)


engagement_overview = (
    engagement_overview.with_columns(CTR=(pos_expr / neg_expr))
    .with_columns(
        CTR_Lift_vs_NBA=(pl.col("CTR") / pl.repeat((pl.col("CTR").filter(pl.col("MktValue").is_null())), pl.count()) - 1.0).over(
            ["pyChannel", "pyDirection"]
        )
    )
    # .with_columns(
    #     CTR_Lift_vs_NBA=(pl.col("CTR")
    #     / (pl.col("CTR").filter(pl.col("MktValue").is_null()))
    #     - 1.0).over(["pyChannel", "pyDirection"])
    # )
    .sort(["pyChannel", "pyDirection", "Experiment"] + optional_mcg)
)


def set_background_col(s, color):
    return "background-color: %s" % color


def set_font_weight(val, weight="bold"):
    return "font-weight: %s" % weight


engagement_overview.to_pandas().style.hide().applymap(
    set_background_col, subset=pd.IndexSlice[:, positive_labels], color="mediumseagreen"
).applymap(
    set_background_col, subset=pd.IndexSlice[:, negative_labels], color="tomato"
).applymap(
    set_background_col,
    subset=pd.IndexSlice[:, ["CTR", "CTR_Lift_vs_NBA"]],
    color="orange",
).applymap(
    set_font_weight, subset=pd.IndexSlice[:, ["pyChannel", "pyDirection", "Experiment"]]
)

# Aggregated over all channels

In [ ]:
import pandas as pd




top_level_dashboard = (
    engagement_overview.group_by(["Experiment", "MktValue"] + optional_mcg)
    .agg(cs.numeric().sum())
    .with_columns(
        (pos_expr / neg_expr).alias("CTR")  # or / pos+neg
    )
    .with_columns(
        CTR_Lift_vs_NBA=pl.col("CTR")
        / (pl.col("CTR").filter(pl.col("MktValue").is_null()))
        - 1.0
    )
    .sort(["Experiment"] + optional_mcg)
)

top_level_dashboard.to_pandas().style.hide().applymap(
    set_background_col, subset=pd.IndexSlice[:, positive_labels], color="mediumseagreen"
).applymap(
    set_background_col, subset=pd.IndexSlice[:, negative_labels], color="tomato"
).applymap(
    set_background_col, subset=pd.IndexSlice[:, ["CTR", "CTR_Lift_vs_NBA"]], color="orange"
).applymap(
    set_font_weight, subset=pd.IndexSlice[:, ["Experiment"]]
)

# Lift

Engagement Lift is calculated as (SuccessRate(test) - SuccessRate(control))/SuccessRate(control)

Value Lift is calculated as (ValueCapture(test) - ValueCapture(control))/ValueCapture(control)

TODO replicate IA tiles